In [ ]:
!wget https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l&dl=0 # train
!wget https://www.dropbox.com/scl/fi/toycvi2k3h0dz42gl3vk0/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7&dl=0 # test
!wget https://www.dropbox.com/scl/fi/khtr33xeykwpjjezlbkdd/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz&dl=0 # validation

--2024-03-14 17:14:15--  https://www.dropbox.com/scl/fi/em9gg01fixe0evr2gm1yr/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca8fb8ea142f681d255046811f9.dl.dropboxusercontent.com/cd/0/inline/CPGQZvMmdUNwsr4cHIQjvkxNUzER0o-JR-FmHPagJrir_2PZrRjmR1nxFIVEdlaX0bXTJc-hd_6-86_yyW6rX1jn7PcaO4p5rKZl1t3AWfDGyqYujOtkX_V9ZvNbUb8f_vc4KXHxv7ykREMj5P5osWq1/file# [following]
--2024-03-14 17:14:16--  https://uca8fb8ea142f681d255046811f9.dl.dropboxusercontent.com/cd/0/inline/CPGQZvMmdUNwsr4cHIQjvkxNUzER0o-JR-FmHPagJrir_2PZrRjmR1nxFIVEdlaX0bXTJc-hd_6-86_yyW6rX1jn7PcaO4p5rKZl1t3AWfDGyqYujOtkX_V9ZvNbUb8f_vc4KXHxv7ykREMj5P5osWq1/file
Resolving uca8fb8ea142f681d255046811f9.dl.dropboxusercontent.com (uca8fb8ea142f681d255046811f9.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:

In [ ]:
# unzip the downloaded zip files
!unzip /content/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l # train
!unzip /content/test.zip?rlkey=6wjn1yvhcfiuymbhkc2ubcro7 # test
!unzip /content/validation.zip?rlkey=zbpzzxv3c368df70nrhssi3sz # validation

Archive:  /content/train.zip?rlkey=82gu4jxpee8ud3xfhkcwobu9l
   creating: train/
  inflating: __MACOSX/._train        
  inflating: train/.DS_Store         
  inflating: __MACOSX/train/._.DS_Store  
   creating: train/flood/
  inflating: __MACOSX/train/._flood  
   creating: train/wildfire/
  inflating: __MACOSX/train/._wildfire  
   creating: train/earthquake/
  inflating: __MACOSX/train/._earthquake  
  inflating: train/flood/63.jpg      
  inflating: __MACOSX/train/flood/._63.jpg  
  inflating: train/flood/189.jpg     
  inflating: __MACOSX/train/flood/._189.jpg  
  inflating: train/flood/77.jpg      
  inflating: __MACOSX/train/flood/._77.jpg  
  inflating: train/flood/162.jpg     
  inflating: __MACOSX/train/flood/._162.jpg  
  inflating: train/flood/176.jpg     
  inflating: __MACOSX/train/flood/._176.jpg  
  inflating: train/flood/88.jpg      
  inflating: __MACOSX/train/flood/._88.jpg  
  inflating: train/flood/348.jpg     
  inflating: __MACOSX/train/flood/._348.jpg  
  inflat

In [ ]:
img_height, img_width, color_channels = 224,224,3

In [ ]:
image_shape = (img_height, img_width)
print(image_shape)

(224, 224)


In [ ]:
input_shape = (img_height, img_width, color_channels)

In [ ]:
train_dir = '/content/train'
test_dir = '/content/test'
valid_dir = '/content/validation'

In [ ]:
batch_size = 32
steps_per_epoch = total_train//32

In [ ]:
import os
total_train = sum([len(files) for r, d, files in os.walk(train_dir)]) - 1
total_test = sum([len(files) for r, d, files in os.walk(test_dir)]) - 1
total_valid = sum([len(files) for r, d, files in os.walk(valid_dir)]) - 1

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
model_head = base_model.output
model_head = GlobalAveragePooling2D()(model_head)
model_head = Dense(128, activation='relu')(model_head)
model_head = Dense(3, activation='softmax')(model_head)

In [ ]:
model = Model(inputs=base_model.input, outputs=model_head)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                   zoom_range=0.15, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.15,
                                   horizontal_flip=True, fill_mode='nearest')

train_gen = train_datagen.flow_from_directory(train_dir, target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 1200 images belonging to 3 classes.


In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                   zoom_range=0.15, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.15,
                                   horizontal_flip=True, fill_mode='nearest')

valid_gen = valid_datagen.flow_from_directory(valid_dir, target_size=image_shape,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 300 images belonging to 3 classes.


In [ ]:
model.fit(train_gen, steps_per_epoch=steps_per_epoch, validation_data=valid_gen,
          validation_steps=total_valid//32, epochs=10 )

Epoch 1/10
37/37 [==============================] - 42s 1s/step - loss: 0.1614 - accuracy: 0.9358 - val_loss: 0.1817 - val_accuracy: 0.9306
Epoch 2/10
37/37 [==============================] - 39s 1s/step - loss: 0.1194 - accuracy: 0.9529 - val_loss: 0.1679 - val_accuracy: 0.9479
Epoch 3/10
37/37 [==============================] - 39s 1s/step - loss: 0.1203 - accuracy: 0.9598 - val_loss: 0.2015 - val_accuracy: 0.9201
Epoch 4/10
37/37 [==============================] - 36s 976ms/step - loss: 0.0979 - accuracy: 0.9692 - val_loss: 0.2337 - val_accuracy: 0.9201
Epoch 5/10
37/37 [==============================] - 36s 984ms/step - loss: 0.0815 - accuracy: 0.9700 - val_loss: 0.1638 - val_accuracy: 0.9340
Epoch 6/10
37/37 [==============================] - 36s 960ms/step - loss: 0.0703 - accuracy: 0.9777 - val_loss: 0.1497 - val_accuracy: 0.9479
Epoch 7/10
37/37 [==============================] - 36s 988ms/step - loss: 0.0544 - accuracy: 0.9769 - val_loss: 0.1592 - val_accuracy: 0.9479
Epoch 8/

In [ ]:
test_gen = train_datagen.flow_from_directory(test_dir,
                                             target_size=image_shape,
                                             batch_size=1,
                                             class_mode='categorical')

test_loss, test_acc = model.evaluate(test_gen,verbose=2)
print(f'test loss: {test_loss} and test accuracy: {test_acc}')

Found 300 images belonging to 3 classes.
300/300 - 8s - loss: 0.2397 - accuracy: 0.9367 - 8s/epoch - 25ms/step
test loss: 0.23970121145248413 and test accuracy: 0.9366666674613953


In [ ]:
model.save('disaster_mobilenetv2_1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
